##### Sentence Completion Using Word2Vec and LSTM

In [ ]:
import gensim
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
import requests
import re

In [ ]:
import requests

r = requests.get("https://ota.bodleian.ox.ac.uk/repository/xmlui/bitstream/handle/20.500.12024/5730/5730.txt")
raw_text = r.text
print(raw_text[0:10000])

TREASURE ISLAND
To S.L.O., an American gentleman in accordance with whose classic taste the following narrative has been designed, it is now, in return for numerous delightful hours, and with the kindest wishes, dedicated by his affectionate friend, the author.
TO THE HESITATING PURCHASER
If sailor tales to sailor tunes,
  Storm and adventure, heat and cold,
If schooners, islands, and maroons,
  And buccaneers, and buried gold,
And all the old romance, retold
  Exactly in the ancient way,
Can please, as me they pleased of old,
  The wiser youngsters of today:
PART ONE—The Old Buccaneer
The Old Sea-dog at the Admiral Benbow
SQUIRE TRELAWNEY, Dr. Livesey, and the rest of these gentlemen having asked me to write down the whole particulars about Treasure Island, from the beginning to the end, keeping nothing back but the bearings of the island, and that only because there is still treasure not yet lifted, I take up my pen in the year of grace 17__ and go back to the time when my father kep

In [ ]:
tokenized_text = gensim.utils.simple_preprocess(raw_text)
print(tokenized_text)
len(tokenized_text)

['treasure', 'island', 'to', 'an', 'american', 'gentleman', 'in', 'accordance', 'with', 'whose', 'classic', 'taste', 'the', 'following', 'narrative', 'has', 'been', 'designed', 'it', 'is', 'now', 'in', 'return', 'for', 'numerous', 'delightful', 'hours', 'and', 'with', 'the', 'kindest', 'wishes', 'dedicated', 'by', 'his', 'affectionate', 'friend', 'the', 'author', 'to', 'the', 'hesitating', 'purchaser', 'if', 'sailor', 'tales', 'to', 'sailor', 'tunes', 'storm', 'and', 'adventure', 'heat', 'and', 'cold', 'if', 'schooners', 'islands', 'and', 'maroons', 'and', 'buccaneers', 'and', 'buried', 'gold', 'and', 'all', 'the', 'old', 'romance', 'retold', 'exactly', 'in', 'the', 'ancient', 'way', 'can', 'please', 'as', 'me', 'they', 'pleased', 'of', 'old', 'the', 'wiser', 'youngsters', 'of', 'today', 'part', 'one', 'the', 'old', 'buccaneer', 'the', 'old', 'sea', 'dog', 'at', 'the', 'admiral', 'benbow', 'squire', 'trelawney', 'dr', 'livesey', 'and', 'the', 'rest', 'of', 'these', 'gentlemen', 'having

65154

In [ ]:
maxlen = 5
step = 1
sentences = []
next_words = []
for i in range(0, len(tokenized_text) - maxlen, step):
    sentences.append(tokenized_text[i: i + maxlen])
    next_words.append(tokenized_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 65149


In [ ]:
import pandas as pd
df = pd.DataFrame({
    'sequence': [' '.join(sentence) for sentence in sentences],
    'sequence_tokens': [ sentence for sentence in sentences],
    'next_word': next_words
})
df

,sequence,sequence_tokens,next_word
0,treasure island to an american,"[treasure, island, to, an, american]",gentleman
1,island to an american gentleman,"[island, to, an, american, gentleman]",in
2,to an american gentleman in,"[to, an, american, gentleman, in]",accordance
3,an american gentleman in accordance,"[an, american, gentleman, in, accordance]",with
4,american gentleman in accordance with,"[american, gentleman, in, accordance, with]",whose
...,...,...,...
65144,ringing in my ears pieces,"[ringing, in, my, ears, pieces]",of
65145,in my ears pieces of,"[in, my, ears, pieces, of]",eight
65146,my ears pieces of eight,"[my, ears, pieces, of, eight]",pieces
65147,ears pieces of eight pieces,"[ears, pieces, of, eight, pieces]",of


In [ ]:
model = gensim.models.Word2Vec(
    df.sequence_tokens,
    window = 3,
    min_count = 2,
    workers=4
)
model.save("./word2vec-sentence-completion.model")

In [ ]:
w2v_model = gensim.models.Word2Vec.load("word2vec-sentence-completion.model")
word_vocab= w2v_model.wv.key_to_index
len(word_vocab)

5830

In [ ]:
word_vocab= w2v_model.wv.key_to_index
word_vocab

{'the': 0,
 'and': 1,
 'of': 2,
 'to': 3,
 'was': 4,
 'you': 5,
 'in': 6,
 'he': 7,
 'that': 8,
 'it': 9,
 'had': 10,
 'his': 11,
 'as': 12,
 'my': 13,
 'with': 14,
 'for': 15,
 'on': 16,
 'we': 17,
 'but': 18,
 'me': 19,
 'at': 20,
 'all': 21,
 'not': 22,
 'said': 23,
 'were': 24,
 'this': 25,
 'by': 26,
 'him': 27,
 'have': 28,
 'one': 29,
 'there': 30,
 'now': 31,
 'so': 32,
 'man': 33,
 'out': 34,
 'they': 35,
 'be': 36,
 'captain': 37,
 'll': 38,
 'silver': 39,
 'from': 40,
 'if': 41,
 'up': 42,
 'like': 43,
 'is': 44,
 'no': 45,
 'here': 46,
 'or': 47,
 'what': 48,
 'when': 49,
 'could': 50,
 'doctor': 51,
 'would': 52,
 'us': 53,
 'then': 54,
 'been': 55,
 'well': 56,
 'upon': 57,
 'down': 58,
 'more': 59,
 'them': 60,
 'see': 61,
 'our': 62,
 'before': 63,
 'an': 64,
 'about': 65,
 'your': 66,
 'time': 67,
 'good': 68,
 'their': 69,
 'again': 70,
 'hand': 71,
 'her': 72,
 'still': 73,
 'old': 74,
 'into': 75,
 'two': 76,
 'long': 77,
 'ship': 78,
 'do': 79,
 'some': 80,
 'who':

In [ ]:
vector_size = w2v_model.vector_size
vector_size

100

In [ ]:
tokenized_text[0]

'treasure'

In [ ]:
w2v_model.wv[tokenized_text[0]]

array([ 1.60063520e-01,  1.21351600e-01,  1.39197797e-01,  1.60101548e-01,
       -2.20771998e-01, -4.68920052e-01,  2.93024659e-01,  5.42618096e-01,
       -9.38627645e-02, -3.98069561e-01, -1.45689860e-01, -1.24708772e-01,
        9.29866284e-02,  1.29272699e-01,  2.57411808e-01, -2.18039230e-01,
        1.45753995e-01, -1.99749783e-01, -5.08331418e-01, -4.56788063e-01,
        7.44034071e-03,  3.00901949e-01,  4.08086121e-01, -5.97182214e-01,
        8.15625414e-02, -4.64397445e-02, -6.23406649e-01,  1.77804351e-01,
       -3.15674424e-01,  1.72528684e-01,  2.23554000e-01,  2.04594687e-01,
        1.99148849e-01, -6.76987052e-01, -2.61488140e-01,  4.87068653e-01,
        5.42942733e-02,  1.98780820e-02,  2.61340203e-04, -4.02623951e-01,
       -1.57968462e-01, -2.81344861e-01, -4.54084963e-01, -8.15846622e-02,
        5.25668979e-01, -4.25866574e-01, -3.26178759e-01, -1.32730544e-01,
        4.22403336e-01,  3.90590519e-01,  1.70352876e-01, -3.43753546e-01,
        6.39831573e-02, -

In [ ]:
w2v_model.wv.similar_by_vector(w2v_model.wv[tokenized_text[0]], topn=3)

[('treasure', 1.0),
 ('cabin', 0.9569464325904846),
 ('knoll', 0.9483422040939331)]

In [ ]:
import numpy as np
import pandas as pd

def get_concatenated_word_vectors(tokens, model):
    word_vectors = []
    for token in tokens:
        if token in model.wv:
            word_vectors.append(model.wv[token])
        else:
            word_vectors.append(np.zeros(model.vector_size))
    return np.array(word_vectors)

In [ ]:
df['sequence_vectors'] = df['sequence_tokens'].apply(lambda tokens: get_concatenated_word_vectors(tokens, w2v_model))

In [ ]:
df['next_word_vectors'] = df['next_word'].apply(model.wv.get_vector)

In [ ]:
df.head()

,sequence,sequence_tokens,next_word,sequence_vectors,next_word_vectors
0,treasure island to an american,"[treasure, island, to, an, american]",gentleman,"[[0.16006352, 0.1213516, 0.1391978, 0.16010155...","[-0.09222162, 0.32051453, 0.04839519, 0.205186..."
1,island to an american gentleman,"[island, to, an, american, gentleman]",in,"[[0.43904346, -0.0284028, 0.20828505, 0.137456...","[-0.16606602, 0.20289525, -0.25014842, 0.13079..."
2,to an american gentleman in,"[to, an, american, gentleman, in]",accordance,"[[-0.71108645, 1.0564255, -0.44698468, 0.35716...","[0.00076052814, 0.04978176, 0.0012926898, 0.03..."
3,an american gentleman in accordance,"[an, american, gentleman, in, accordance]",with,"[[-0.45079443, 0.28610468, -0.16750827, 0.4985...","[-0.49365145, 0.14927989, -0.41491613, 0.16131..."
4,american gentleman in accordance with,"[american, gentleman, in, accordance, with]",whose,"[[0.0024640248, 0.02314254, 0.006971939, 0.026...","[-0.047689803, 0.14236933, 0.020593273, 0.1295..."


In [ ]:
df.sequence_vectors[0].shape

(5, 100)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[["sequence","sequence_vectors"]],
                                                        df[["next_word","next_word_vectors"]],
                                                        test_size=0.2,random_state=42)

In [ ]:
X_train.sequence_vectors[0].shape

(5, 100)

In [ ]:
print('Building model...')
model = Sequential()

# First LSTM layer: expects input of shape (sequence_length, vector_size)
model.add(LSTM(128, input_shape=(5, 100), return_sequences=True))

# Second LSTM layer: output will be the final state (not a sequence)
model.add(LSTM(16, return_sequences=False))

# Output layer: predicting the next word's vector (100-dimensional vector)
model.add(Dense(100, activation='linear'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=optimizer)

model.summary()

Building model...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 5, 128)              │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           9,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 100)                 │           1,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 128,228 (500.89 KB)

 Trainable params: 128,228 (500.89 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_train_sequence_vectors = np.array([np.array(seq) for seq in X_train.sequence_vectors])
y_train_next_word_vectors = np.array([np.array(seq) for seq in y_train.next_word_vectors])
X_test_sequence_vectors = np.array([np.array(seq) for seq in X_test.sequence_vectors])
y_test_next_word_vectors = np.array([np.array(seq) for seq in y_test.next_word_vectors])

In [ ]:
model.fit(X_train_sequence_vectors, y_train_next_word_vectors, batch_size=128, epochs=60)

Epoch 1/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 15s 29ms/step - loss: 0.0808
Epoch 2/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - loss: 0.0738
Epoch 3/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0728
Epoch 4/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0718
Epoch 5/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - loss: 0.0714
Epoch 6/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0706
Epoch 7/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - loss: 0.0705
Epoch 8/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - loss: 0.0705
Epoch 9/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0701
Epoch 10/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - loss: 0.0703
Epoch 11/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 0.0698
Epoch 12/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 0.0692
Epoch 13/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - loss: 0.0685
Epoch 14/60
408/408 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0691
Epoch 15/60
408/408 ━━━━━━━━━

In [ ]:
import numpy as np

def get_concatenated_word_vectors(tokens, model):
    word_vectors = []
    for token in tokens:
        if token in model.wv:
            word_vectors.append(model.wv[token])
        else:
            word_vectors.append(np.zeros(model.vector_size))
    return np.array(word_vectors)

def predict_next_word(sequence, model, w2v_model, top_n=1):
    sequence_vectors = get_concatenated_word_vectors(sequence, w2v_model)
    sequence_vectors = np.expand_dims(sequence_vectors, axis=0)
    predicted_vector = model.predict(sequence_vectors)
    predicted_word = w2v_model.wv.similar_by_vector(predicted_vector[0], topn=top_n)
    predicted_words = [word for word, _ in predicted_word]
    return predicted_words

def generate_text(initial_sequence, model, w2v_model, num_words=50):
    sequence = gensim.utils.simple_preprocess(initial_sequence)
    generated_text = sequence[:]

    for _ in range(num_words):
        predicted_words = predict_next_word(generated_text[-5:], model, w2v_model, top_n=1)
        generated_text.append(predicted_words[0])

    return ' '.join(generated_text)

initial_sequence = "the rest of these gentlemen"
generated_text = generate_text(initial_sequence, model, w2v_model, num_words=5)
print(generated_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
the rest of these gentlemen pleased also fortune saying yes
